In [80]:
import pandas as pd
import json
import re
import os
import subprocess
import bibtexparser
from tqdm import tqdm

In [ ]:
#TODO: write a function that normalizes the names of the authors from our annotations
# to fit the format generated by anystyle.io check below for examples and look for the pattern

In [75]:
folder_path = "../all_data_articles"

for filename in os.listdir(folder_path):
    if filename.endswith(".json") and filename.startswith("Royal_Companies"):
        file_path = os.path.join(folder_path, filename)
        article = json.load(open(file_path, "r"))
        
        extraction = {}

        for number, footnote in article["footnotes"].items():
            references = re.split(';',footnote)
            
            author_title_list = []

            for reference in references:
                command = ['ruby', 'anystyle.rb', str(reference)]
                bibtex = subprocess.run(command, stdout=subprocess.PIPE, text=True).stdout
                parsed_bibtex = bibtexparser.loads(bibtex).entries[0]

                #print(parsed_bibtex)

                # Extract title, prioritizing 'booktitle' if both 'title' and 'booktitle' are present
                title = parsed_bibtex.get('booktitle', parsed_bibtex.get('title', None))

                # Extract author information
                author = parsed_bibtex.get('author', None)

                # Append author and title pair to the list
                author_title_list.append([author, title])

            # Store the list in the extraction dictionary with the footnote number as the key
            extraction[number] = author_title_list

            if number == "8":
                break
                

In [76]:
extraction 

{'1': [['Paris',
   'Z/1d/85, fo. 1v, summary register documenting the policy on the Amitié'],
  [None,
   'fos 4v–6, record of the arbitration dispute between the two companies']],
 '2': [[None,
   'The key register from the Royal Insurance Company that is used is its arbitration register: AN, Z/1d/84. The key bundle of documents for the Parisian admiralty court is AN, Z/1d/109']],
 '3': [[None, 'Marine Insurance: Origins and Institutions, 1300–1850'],
  ['Kingston, C.',
   'Governance and Institutional Change in Marine Insurance, 1350–1850’']],
 '4': [[None,
   'The only semi-extensive treatment of the Royal Insurance Company in the past century has been Louis-Augustin Boiteux’s brief and imbalanced study in L.-A. Boiteux, L’assurance maritime à Paris sous le règne de Louis XIV']],
 '5': [['Wubs-Mrozewicz, J.',
   'Conflict Management and Interdisciplinary History: Presentation of a New Project and an Analytical Model’'],
  [None,
   'Conflict Management in the Mediterranean and the 

loading the annotated data and converting it to the set of triplets

In [56]:
def file_finder(file_str: str) -> str:
    """
    This function takes a file name and returns the path to the file in the all_data_articles.
    """
    title_doi = "../data/titles_doi.csv"
    folder_path = "../all_data_articles"
    
    #extract the doi from the file name
    doi = file_str.split("_")[-1].split(".")[0]

    # find the row in the csv file where the doi column ends with the doi
    df = pd.read_csv(title_doi)
    doi_row = df[df["DOI"].str.endswith(doi)]

    # extract the title from the row
    title_json = doi_row["Title"].values[0].replace(" ", "_") + ".json"

    for filename in os.listdir(folder_path):
        if filename.endswith(".json") and filename.startswith(title_json[:int(len(title_json)/3)]):
            return filename


In [109]:
def load_annotations(file_str: str) -> pd.DataFrame:
    """
    This function takes a file name and returns the annotations from the file.
    And also replaces missing values with None.
    """
    folder_path = "../data/annotated"

    file_path = os.path.join(folder_path, file_str)
    df = pd.read_excel(file_path)

    # replace missing values with None
    df = df.where(pd.notnull(df), None)

    # replace values marked with nan with None
    df = df.replace("nan", None)
    
    return df

In [178]:
def format_author_name(name):
    # TODO: probably need to handle more cases
    if name is None:
        return None
    if ' and ' in name:
        # Handle multiple authors
        authors = name.split(' and ')
        formatted_authors = [format_author_name(author) for author in authors]
        return ' and '.join(formatted_authors)
    else:
        parts = name.split()
        # Handle case where there is a middle initial
        if len(parts) == 3:
            return f"{parts[1]}, {parts[0]} {parts[2]}"
        # Handle case where there is no middle initial
        elif len(parts) == 2:
            return f"{parts[1]}, {parts[0]}"
        else:
            return name

In [169]:
def df_to_triplets(df: pd.DataFrame) -> set:
    """
    This function takes a dataframe and returns a set of triplets.
    """
    triplets = set()
    for i in range(len(df)):
        triplet = (df.iloc[i]["Footnote"], format_author_name(df.iloc[i]["Authors"]), df.iloc[i]["Title"])
        triplets.add(triplet)
    return triplets

In [170]:
def dict_to_triplets(extraction: dict) -> set:
    """
    Converts a dictionary of footnotes to a set of triplets
    """
    triplets = set()

    for number, references in extraction.items():
        for reference in references:
            author = reference[0]
            title = reference[1]

            if author == "":
                author = None

            if title == "":
                title = None
            
            triplets.add((int(number), author, title))

    return triplets

In [171]:
def information_extraction(file_path: str) -> set:
    """
    This function takes a file path and returns a set of triplets.
    """
    article = json.load(open(file_path, "r"))
    extraction = {}

    prev_footnote = None

    for number, footnote in tqdm(article["footnotes"].items()):

        # If the footnote is ibid, use the previous footnote
        if footnote.startswith("Ibid"):
            footnote = prev_footnote

        # Store the footnote for the next iteration
        prev_footnote = footnote
        
        references = re.split(';',footnote)
        
        author_title_list = []

        for reference in references:

            command = ['ruby', 'anystyle.rb', str(reference)]
            bibtex = subprocess.run(command, stdout=subprocess.PIPE, text=True).stdout
            parsed_bibtex = bibtexparser.loads(bibtex).entries[0]

            #print(parsed_bibtex)

            # Extract title, prioritizing 'booktitle' if both 'title' and 'booktitle' are present
            title = parsed_bibtex.get('booktitle', parsed_bibtex.get('title', None))

            # Extract author information
            author = parsed_bibtex.get('author', None)

            # Append author and title pair to the list
            author_title_list.append([author, title])

        # Store the list in the extraction dictionary with the footnote number as the key
        extraction[number] = author_title_list

    return dict_to_triplets(extraction)

In [180]:
def calculate_scores(triplets: set, extraction: set) -> dict:
    """
    This function takes two sets of triplets and returns a dictionary of scores.
    """
    scores = {"TP": 0, "FP": 0, "FN": 0}

    for triplet in triplets:
        if triplet in extraction:
            scores["TP"] += 1
        else:
            scores["FN"] += 1

    for triplet in extraction:
        if triplet not in triplets:
            scores["FP"] += 1

    return scores

In [183]:
def calculate_scores(triplets, extractions):
    TP = len(triplets & extractions)  # Intersection of triplets and extractions
    FP = len(extractions - triplets)  # Elements in extractions but not in triplets
    FN = len(triplets - extractions)  # Elements in triplets but not in extractions

    recall = TP / (TP + FN) if TP + FN != 0 else 0
    precision = TP / (TP + FP) if TP + FP != 0 else 0
    f_score = 2 * (precision * recall) / (precision + recall) if precision + recall != 0 else 0

    return recall, precision, f_score

In [184]:
calculate_scores(triplets, extraction)

(0.2268041237113402, 0.2430939226519337, 0.23466666666666663)

In [189]:
path_annotations = "../data/annotated"

for filename in os.listdir(path_annotations):
    print(filename)
    title_json = file_finder(filename)
    #file_path = os.path.join(folder_path, title_json)
    #article = json.load(open(file_path, "r"))
    df = load_annotations(filename)
    triplets = df_to_triplets(df)
    extraction = information_extraction(file_path)
    recall, precision, f_score = calculate_scores(triplets, extraction)
    print(f"Recall: {recall}")
    print(f"Precision: {precision}")
    print(f"F-Score: {f_score}")
    print("-"* 25)
    print("\n")

Labels - https___doi.org_10.1093_ehr_cew052.xlsx


 10%|█         | 15/148 [00:05<00:49,  2.71it/s]

In [182]:
def extract_triplets(triplets):
    return list(filter(lambda triplet: 30 < triplet[0] < 40, triplets))

In [148]:
extract_triplets(extraction)

[(32, 'Madge', 'Press, Radio, and Social Consciousness’'),
 (38, 'Madge and Harrisson, Mass Observation', None),
 (37, None, 'First Year’s Work, 1937–1938'),
 (31, 'Warner, R.', 'Mind in Chains'),
 (33, 'Madge and Harrisson, Mass Observation', None),
 (36, None, 'First Year’s Work, 1937–1938'),
 (39, 'Madge', 'Press, Radio, and Social Consciousness’'),
 (35,
  'Jahoda-Lazarsfeld, M. and Zeisel, H.',
  'Die Arbeitslosen von Marienthal'),
 (34,
  'Lynd, R.S. and Lynd, H.M. and Middletown and Igo, S.',
  'For the best overview of the study'),
 (35, 'Madge and Harrisson, Mass Observation', None)]

In [149]:
extract_triplets(triplets)

[(32, 'Madge', 'Press, Radio, and Social Consciousness’'),
 (37, 'C. Madge and T. Harrisson', 'First Year’s Work, 1937–1938'),
 (34,
  'S. Igo',
  'The Averaged American: Surveys, Citizens, and the Making of a Mass Public'),
 (34, 'R.S. Lynd and H.M. Lynd', 'Middletown'),
 (36, 'C. Madge and T. Harrisson', 'First Year’s Work, 1937–1938'),
 (31, 'R. Warner', 'Education'),
 (33, 'Madge and Harrisson', 'Mass Observation'),
 (39, 'Madge', 'Press, Radio, and Social Consciousness'),
 (35, 'M. Jahoda-Lazarsfeld and H. Zeisel', 'Die Arbeitslosen von Marienthal'),
 (38, 'Madge and Harrisson', 'Mass Observation')]

In [ ]:
#(113, 'M. Young and P. Willmott', 'Family and Kinship in East London'),
#('113', 'Young, M. and Willmott, P.', 'Family and Kinship in East London'),